In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import time
import splitfolders
import shutil
import random
import csv
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator

main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
dataset_dir = os.path.join(main_dir, 'image_dataset')
dataset_dir_2 = os.path.join(main_dir, 'image_dataset_2')
train_dir = os.path.join(dataset_dir_2, 'train')
val_dir = os.path.join(dataset_dir_2, 'val')
test_dir = os.path.join(dataset_dir_2, 'test')

# splits the image dataset into 64% training, 16% validation, and 20% test sets into a new directory
def split_images():
    randNum = random.randint(0, 1337)
    if os.path.exists(dataset_dir_2):
        shutil.rmtree(dataset_dir_2)
    if not os.path.exists(dataset_dir_2):
        os.mkdir(dataset_dir_2)
    splitfolders.ratio(dataset_dir, output=dataset_dir_2, 
                       seed=randNum, ratio=(.64, .16, .2), 
                       group_prefix=None)
    return "Dataset splitting completed"

# saves the training record into csv file for comparison
def save_training_record(model_name, attempt):
    csv_path = os.path.join(main_dir, 'imageClassificationsComparisons.csv')
    csv_file = open(csv_path, 'a', newline='')
    wr = csv.writer(csv_file)
    wr.writerow([model_name, attempt, initial_epochs, earlystopping.stopped_epoch+1-initial_epochs, 
                 training_time, loss1*100, accuracy1*100, 60*accuracy1])
    csv_file.close()
    return "record saved in csv file successfully"

In [2]:
#Splits the dataset
split_images()

Copying files: 291 files [00:00, 610.06 files/s]


'Dataset splitting completed'

In [5]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )


test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                           batch_size=60,
                                                          image_size=IMG_SIZE)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))
validation_generator = test_datagen.flow_from_directory(val_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))


Found 60 files belonging to 2 classes.
Found 185 images belonging to 2 classes.
Found 46 images belonging to 2 classes.


In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(15, 15))
for images, labels in train_dataset.take(1):
  for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Configure the Dataset For Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

Rescaling Pixel Values

# Creating Base Model from Inception

In [7]:
# Create the base model from the pre-trained model InceptionV3
IMG_SHAPE = IMG_SIZE + (3,)
base_model = InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')




In [8]:
### Freeze The Convolutional Base
#It is important to freeze the convolutional base before you compile and train the model. 
#Freezing (by setting layer.trainable = False) prevents the weights in a given layer from 
#being updated during training. MobileNet V2 has many layers, so setting the entire model's 
#trainable flag to False will freeze all of them.

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
### Important note about BatchNormalization layers

#Many models contain `tf.keras.layers.BatchNormalization` layers. This layer is a special case and precautions 
#should be taken in the context of fine-tuning, as shown later in this tutorial. 

#When you set `layer.trainable = False`, the `BatchNormalization` layer will run in inference mode, and will 
#not update its mean and variance statistics. 

#When you unfreeze a model that contains BatchNormalization layers in order to do fine-tuning, you should keep 
#the BatchNormalization layers in inference mode by passing `training = False` when calling the base model. 
#Otherwise, the updates applied to the non-trainable weights will destroy what the model has learned.

#For more details, see the [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).

# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
### Add a classification head

#To generate predictions from the block of features, average over the spatial `5x5` spatial locations, using a `tf.keras.layers.GlobalAveragePooling2D` layer to convert the features to  a single 1280-element vector per image.

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
# Keras Dense layer is added to convert features into a single predictions per image, 
# in which positive numbers predict class 1 (Normal Roads) and negative numbers predict
# no activation is required as predictions will be a logit or raw prediction value

prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [9]:
#Build a model by chaining together the data augmentation, rescaling, `base_model` and feature extractor 
#layers using the [Keras Functional API](https://www.tensorflow.org/guide/keras/functional). As previously mentioned, 
#use `training=False` as our model contains a `BatchNormalization` layer.


from tensorflow.keras.optimizers import RMSprop

x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

# Feature Extraction

In [ ]:
### Compile the model

#Compile the model before training it. Since there are two classes, use the `tf.keras.losses.BinaryCrossentropy` 
#loss with `from_logits=True` since the model provides a linear output.

base_learning_rate = 0.00012
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

#The 2.5 million parameters in MobileNet are frozen, but there are 1.2 thousand _trainable_ parameters in the Dense layer. 
#These are divided between two `tf.Variable` objects, the weights and biases.

len(model.trainable_variables)

In [10]:
### Train the model with earlystopping callbacks to avoid overfitting the model. 
# Usually earlystopping will be called in the Fine Tuning phase instead of Feature Extraction

from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

initial_epochs = 50

start_time = time.time()


inc_history = model.fit(train_generator, validation_data = validation_generator, epochs = 50)


Epoch 1/50
10/10 [==============================] - 27s 2s/step - loss: 10.5684 - acc: 0.5730 - val_loss: 1.3472 - val_acc: 0.4565
Epoch 2/50
10/10 [==============================] - 18s 2s/step - loss: 0.6125 - acc: 0.7622 - val_loss: 0.8984 - val_acc: 0.6957
Epoch 3/50
10/10 [==============================] - 19s 2s/step - loss: 0.7297 - acc: 0.7243 - val_loss: 2.4698 - val_acc: 0.6522
Epoch 4/50
10/10 [==============================] - 19s 2s/step - loss: 0.5034 - acc: 0.8486 - val_loss: 0.4435 - val_acc: 0.8478
Epoch 5/50
10/10 [==============================] - 18s 2s/step - loss: 0.7445 - acc: 0.7838 - val_loss: 0.8903 - val_acc: 0.8043
Epoch 6/50
10/10 [==============================] - 18s 2s/step - loss: 0.5547 - acc: 0.8108 - val_loss: 0.8414 - val_acc: 0.8261
Epoch 7/50
10/10 [==============================] - 20s 2s/step - loss: 0.5252 - acc: 0.8324 - val_loss: 0.2631 - val_acc: 0.8696
Epoch 8/50
10/10 [==============================] - 18s 2s/step - loss: 0.4592 - acc: 0.8

# Fine tuning

In [ ]:
#In the feature extraction experiment, you were only training a few layers on top of an MobileNetV2 base model. 
#The weights of the pre-trained network were **not** updated during training.

#One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the 
#pre-trained model alongside the training of the classifier you added. The training process will force the weights 
#to be tuned from generic feature maps to features associated specifically with the dataset.

#Note: This should only be attempted after you have trained the top-level classifier with the pre-trained model set 
#to non-trainable. If you add a randomly initialized classifier on top of a pre-trained model and attempt to train 
#all layers jointly, the magnitude of the gradient updates will be too large (due to the random weights from the classifier) 
#and your pre-trained model will forget what it has learned.

#Also, you should try to fine-tune a small number of top layers rather than the whole MobileNet model. In most convolutional 
#networks, the higher up a layer is, the more specialized it is. The first few layers learn very simple and generic features 
#that generalize to almost all types of images. As you go higher up, the features are increasingly more specific to the dataset 
#on which the model was trained. The goal of fine-tuning is to adapt these specialized features to work with the new dataset, 
#rather than overwrite the generic learning.

### Un-freeze the top layers of the model


#All you need to do is unfreeze the `base_model` and set the bottom layers to be un-trainable. Then, you should recompile the 
#model (necessary for these changes to take effect), and resume training.

base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 5

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
### Compile the model

#As you are training a much larger model and want to readapt the pretrained weights, it is important to use a lower 
#learning rate at this stage. Otherwise, your model could overfit very quickly.
ft_learning_rate = 0.000008

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=ft_learning_rate),
              metrics=['accuracy'])

model.summary()

len(model.trainable_variables)

In [ ]:
### Continue training the model

#If you trained to convergence earlier, this step will improve your accuracy by a few percentage points.

fine_tune_epochs = 50
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=(history.epoch[-1]+1),
                         validation_data=validation_dataset,
                         callbacks =[earlystopping])



In [ ]:
#Let's take a look at the learning curves of the training and validation accuracy/loss when fine-tuning the last few 
#layers of the MobileNetV2 base model and training the classifier on top of it. The validation loss is much higher than the 
#training loss, so you may get some overfitting.

#You may also get some overfitting as the new training set is relatively small and similar to the original MobileNetV2 datasets.

training_time = time.time() - start_time
print("--- %s seconds taken to train model ---" % (training_time))

acc = history.history['accuracy']
acc += history_fine.history['accuracy']

val_acc = history.history['val_accuracy']
val_acc += history_fine.history['val_accuracy']

loss = history.history['loss']
loss += history_fine.history['loss']

val_loss = history.history['val_loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.3, 1])
plt.plot([initial_epochs+1,initial_epochs+1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs+1,initial_epochs+1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [12]:
### Evaluation and prediction

#Finally you can verify the performance of the model on new data using test set.

loss1, accuracy1 = model.evaluate(test_dataset)
print("The test accuracy is {} and loss is {}".format(accuracy1, loss1))

1/1 [==============================] - 3s 3s/step - loss: 106.0931 - acc: 0.3667
The test accuracy is 0.36666667461395264 and loss is 106.09306335449219


In [ ]:
#And now you are all set to use this model to predict if your pet is a normal or accident.

# Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(15, 15))
for i in range(60):
  ax = plt.subplot(8, 8, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

In [ ]:
print("FE Epoch Undergo: ", initial_epochs)
print("FT Epoch Undergo: ", earlystopping.stopped_epoch+1 - initial_epochs)
print("Loss Percentage: %.2f" % (loss1*100))
print("Accuracy Percentage: %.2f" % (accuracy1*100))
print("Training Time %s seconds" % (training_time))
print("Correct Predictions out of 60: ", int(60*accuracy1))

save_training_record("InceptionV3", 5)

In [ ]:
#models_dir = os.path.join(main_dir, 'saved_models')
#inception_model = os.path.join(models_dir, 'InceptionV3_2')
#model.save(inception_model)